# Scipy Intro

`scipy` is a close partner module to `numpy`, but while `numpy` focuses on the interchangable functionality, `scipy` builds on top of that is targeted to scientific applications in mathematics, science and engineering. It provides functionality for

 * numerical integration
 * interpolation
 * optimization
 * linear algebra
 * statistics
 
In practice the distinction is a bit blurry (e.g. numpy has functionality for FFTs) but since they share a common heritage they're so similar to use it shouldn't cause a problem.

## Linear Algebra

The `linalg` submodule contains functions for most of the common linear algebra operations (mmult, det, etc.). Underneath it relies on BLAS/LAPACK on numpy `ndarray`s to be fast

In [ ]:
import numpy as np
from scipy import linalg

### Determinants and Inverses

In [ ]:
arr = np.array([[1, 2],
                [3, 4]])

In [ ]:
linalg.det(arr)

In [ ]:
iarr = linalg.inv(arr)
iarr

In [ ]:
iarr * arr

In [ ]:
np.dot(iarr, arr)

### Solving linear systems


In [ ]:
a = np.array([[3, 2, 0], [1, -1, 0], [0, 5, 1]])
b = np.array([2, 4, -1])

In [ ]:
x = linalg.solve(a, b)
x

In [ ]:
np.dot(a, x) == b

There are also `solve_banded`, `solve_circulant`, `solve_triangular`, `solve_toeplitz` for more specific cases. If you're interested in solving systems exactly (i.e. not numerically, check out [sympy](https://sympy.org) and [sage](https://www.sagemath.org).

### Eigenvalues & Eigenvectors


In [ ]:
a = np.array([[0., -1.], [1., 0.]])

In [ ]:
linalg.eigvals(a)

In [ ]:
evals, evects = linalg.eig(a)

In [ ]:
evals

In [ ]:
evects

In [ ]:
np.dot(a, evects[0])

Like the other functions, there are more specific versions of `.eig` (hermition, banded, ...), try `linalg.eig<TAB>`

### Decompositions

There are methods for LU-decomposition, SVD, Cholesky, qr etc.

In [ ]:
m, n = 9, 6
a = np.random.randn(m, n) + 1.j*np.random.randn(m, n)
U, s, Vh = linalg.svd(a)
U.shape,  s.shape, Vh.shape

In [ ]:
sigma = np.zeros((m, n))
for i in range(min(m, n)):
    sigma[i, i] = s[i]
    
a1 = np.dot(U, np.dot(sigma, Vh))
np.allclose(a, a1)

## Interpolation

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
x = np.linspace(0, 2*np.pi, 10)

noise = (np.random.random(10)*2 - 1) * 1e-1

y = np.sin(x)

In [ ]:
linear_interp = interp1d(x, y+noise)

In [ ]:
type(linear_interp)

This is a function which we can feed valus to to have the function evaluated at those points. We can also pass the 'kind' keyword to change the interpolation type

In [ ]:
cubic_interp = interp1d(x, y+noise, kind='cubic')

In [ ]:
xintp = np.linspace(0, 2*np.pi, 100)
yintp_lin = linear_interp(xintp)
yintp_cub = cubic_interp(xintp)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline 
plt.rcParams['figure.figsize'] = (16, 12)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x, y, noise, capsize=15, capthick=2.0, fmt='.', color='k')
ax.plot(xintp, yintp_lin, label='linear')
ax.plot(xintp, yintp_cub, label='cubic')
ax.legend()

## Optimization

This submodule handles general optimization problems. Curve fitting is one example. Suppose we have the following lists of extreme monthly temperatures for somewhere in Alaska (this example is taken from [this tutorial](https://scipy-lectures.org/intro/scipy/auto_examples/solutions/plot_curvefit_temperature_data.html#sphx-glr-intro-scipy-auto-examples-solutions-plot-curvefit-temperature-data-py)

In [ ]:
from scipy import optimize

t_max = [ 17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18 ]
t_min = [ -62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58 ]

Can we fit these to some function? First look at the data to get some inspiration

In [ ]:
fig, ax = plt.subplots()

ax.plot(t_max, label="Maximum monthly temperature", color='r', marker = '+', markersize=14, linestyle="None")
ax.plot(t_min, label="Minimum monthly temparature", color='b', marker = '+', markersize=14, linestyle="None")
ax.legend()

In [ ]:
def yearly_temps(times, avg, ampl, time_offset):
    return (avg + ampl * np.cos((times + time_offset) * 2 * np.pi / times.max()))

In [ ]:
months = np.arange(12)
days = np.linspace(0, 12, num=365)

res_max, cov_max = optimize.curve_fit(
    yearly_temps, months, t_max, [20, 10, 0]
)

res_min, cov_min = optimize.curve_fit(
    yearly_temps, months, t_min, [-40, 20, 0]
)

In [ ]:
fig, ax = plt.subplots()

ax.plot(months, t_max, label="Maximum monthly temperature", color='r', marker = '+', markersize=14, linestyle="None")
# Remember *res_max will expand the list into separate arguments
ax.plot(days, yearly_temps(days, *res_max), 'r-')

ax.plot(months, t_min, label="Minimum monthly temparature", color='b', marker = '+', markersize=14, linestyle="None")
ax.plot(days, yearly_temps(days, *res_min), 'b-')


## Numerical Integration

Scipy includes numerical integration routines for numerical integration. (Again, if you are looking for symbolic integration or have to deal with nasty functions look into sympy and sage or other more specific modules (e.g. for MC).

In [ ]:
from scipy import integrate

In [ ]:
invexp = lambda x: np.exp(-x)

In [ ]:
integrate.quad(invexp, 0, np.inf)

In [ ]:
def exy(x, y):
    return np.exp(-(x**2 + y**2))

In [ ]:
exy = lambda x, y:  np.exp(-(x**2 + y**2))


In [ ]:
integrate.dblquad(exy, -10, 10, lambda x: -10, lambda x: 10)